In [ ]:
# load R packages
library(readxl)
packageVersion('readxl')
library(dplyr)
packageVersion('dplyr')
library(stringr)
packageVersion('stringr')
library(fastDummies)
packageVersion('fastDummies')
library(tidyr)
packageVersion('tidyr')
library(lubridate)
packageVersion('lubridate')
library(ggplot2)
packageVersion('ggplot2')

In [ ]:
# set directory
project.dir = '...'
data.dir = '...'
regeps.dir = '...'
raw.rpdr.dir = file.path(regeps.dir, '...')
cleaned.rpdr.dir = file.path(regeps.dir, '...')

In [ ]:
# check lab file to see how many rows
lab.lines <-readLines(file.path(raw.rpdr.dir, "Lab.txt"))
length(lab.lines)

In [ ]:
# load lab file
lab.data <- read.delim(file.path(raw.rpdr.dir, "Lab.txt"), sep = '|')
dim(lab.data)
length(unique(lab.data$EMPI)) # only 902 indviduals
head(lab.data)

In [ ]:
# extract biobank ID from Subject_Id.csv
data.id <- read.csv(file.path(cleaned.rpdr.dir, 'Subject_Id.csv'))
dim(data.id)

In [ ]:
# merge biobank.ID and phy.data file
lab.data.ID <- merge(lab.data, data.id[,c('Subject_Id', 'EMPI')], by = 'EMPI')
dim(lab.data.ID)
colSums(is.na(lab.data.ID))
head(lab.data.ID)

In [ ]:
table(lab.data.ID$Group_Id)

In [ ]:
acth.data <- lab.data.ID %>% filter(str_detect(Group_Id, 'ACTH'))
table(acth.data $Group_Id)
length(unique(acth.data$Subject_Id)) # only 269 patients having ACTH --> use cortisol data

In [ ]:
# subset data with CORTISOL/Cortisol.cortisol in Test_Description
cor.data <- lab.data.ID %>% filter(str_detect(Test_Description, 'CORTISOL|Cortisol|cortisol'))
dim(cor.data)

# make sure Test_Description has only cortisol
unique(cor.data$Test_Description)

In [ ]:
table(cor.data$Group_Id) # remove ACTH

In [ ]:
# select only cortisol, no ACTH
CORT.group.id <- cor.data %>% filter(Group_Id != 'CORT-ACTH30')
dim(CORT.group.id)
head(CORT.group.id)

unique(CORT.group.id$Test_Description)

In [ ]:
# select columns
CORT.group.id.selected.cols <- CORT.group.id %>% select(Subject_Id, Seq_Date_Time, 
                                                        Group_Id, Result, Reference_Units, 
                                                        Result_Text, Reference_Range, Abnormal_Flag)
dim(CORT.group.id.selected.cols)

In [ ]:
colnames(CORT.group.id.selected.cols)

In [ ]:
# check duplication
CORT.group.id.selected.cols[duplicated(CORT.group.id.selected.cols), ] 

In [ ]:
# remove duplicated rows
CORT.group.id.no.dup <- CORT.group.id.selected.cols[!duplicated(CORT.group.id.selected.cols), ] 
dim(CORT.group.id.no.dup) # 5 duplicated rows

## Clean cortisol result

In [ ]:
sort(CORT.group.id.no.dup$Result) # result has both string, numeric and less than

In [ ]:
# check cortisol value with less than ...
result.w.lt <- CORT.group.id.no.dup %>% filter(str_detect(Result, '<')) # a dataset has only less than..in result
dim(result.w.lt) # 68
head(result.w.lt)

In [ ]:
# remove less than
result.w.lt$Result.new <- gsub('<','',result.w.lt$Result)
dim(result.w.lt)
head(result.w.lt)

In [ ]:
# minus 0.01
result.w.lt$Result.new <- as.numeric(result.w.lt$Result.new)
result.w.lt$Result.new <- result.w.lt$Result.new - 0.01
dim(result.w.lt)
head(result.w.lt, 10)

In [ ]:
table(result.w.lt$Reference_Units) # 9 samples with ng/dL -> convert values

In [ ]:
unique((result.w.lt %>% filter(Reference_Units == 'ng/dL'))$Result.new) # all 49.99

In [ ]:
result.w.lt$Result.new[result.w.lt$Reference_Units == 'ng/dL'] <- 49.99 * 0.001
unique((result.w.lt %>% filter(Reference_Units == 'ng/dL'))$Result.new)

In [ ]:
colnames(result.w.lt)

In [ ]:
# change column name: Result.new to Result.ug.dl
colnames(result.w.lt)[9] <- 'Result.ug.dl'
dim(result.w.lt)
head(result.w.lt)

In [ ]:
# convert string to numeric. auto convert strings to NAs
CORT.group.id.no.dup$Result <- as.numeric(CORT.group.id.no.dup$Result)
dim(CORT.group.id.no.dup)

colSums(is.na(CORT.group.id.no.dup)) # 138 NAs because they are convereted from string to NA
summary(CORT.group.id.no.dup$Result) 

In [ ]:
# change unit
unique(CORT.group.id.no.dup$Reference_Units) # becareful with different units -> need to convert to ug/dL

In [ ]:
# check missing unit values
CORT.group.id.no.dup %>% filter(Reference_Units == '') # empty value
CORT.group.id.no.dup %>% filter(Reference_Units == 'not reported') # empty value

In [ ]:
# convert other units to ug/dL. mcg is same with ug
mcg.or.ug.mL <- CORT.group.id.no.dup %>% filter(Reference_Units == 'mcg/mL' | Reference_Units == 'ug/ml' 
                                                 | Reference_Units == 'ug/mL' | Reference_Units == 'UG/ML')
dim(mcg.or.ug.mL) # 9 rows
table(mcg.or.ug.mL$Reference_Units)
head(mcg.or.ug.mL)

mcg.or.ug.mL$Result.ug.dl <- mcg.or.ug.mL$Result * 100  # convert mcg/mL to ug/dL
dim(mcg.or.ug.mL)
mcg.or.ug.mL

In [ ]:
# convert ng/dl to ug/dl. 1ng/dL to 0.001 ug/dl
ng.dl <- CORT.group.id.no.dup %>% filter(Reference_Units == 'ng/dL')
dim(ng.dl) # 55
head(ng.dl)

ng.dl$Result.ug.dl <- ng.dl$Result * 0.001  # convert ng/dl to ug/dL
dim(ng.dl)
head(ng.dl)

In [ ]:
# combine two files ng/dl and mcg/ml
non.ug.dl <- rbind(ng.dl, mcg.or.ug.mL)
dim(non.ug.dl)
head(non.ug.dl)

In [ ]:
# select only ud.dl from CORT.group.id.no.dup
ug.dl <- CORT.group.id.no.dup %>% filter(Reference_Units == 'ug/dL' | Reference_Units == 'ug/dl' 
                                           | Reference_Units == 'UG/DL' | Reference_Units == 'mcg/dL' 
                                           | Reference_Units == 'MCG/DL')

table(ug.dl$Reference_Units)
ug.dl$Result.ug.dl <- ug.dl$Result
dim(ug.dl)
head(ug.dl)

In [ ]:
# combine files after converting units
final.cor.values <- rbind(non.ug.dl, ug.dl)
head(final.cor.values)
dim(final.cor.values)

In [ ]:
length(unique(final.cor.values$Subject_Id)) # 899 ind
colSums(is.na(final.cor.values))

In [ ]:
final.cor.values.no.missing <- final.cor.values %>% drop_na() # remove 133 missing values
dim(final.cor.values.no.missing)
head(final.cor.values.no.missing)
length(unique(final.cor.values.no.missing$Subject_Id)) # 898 ind

In [ ]:
# check high in Abnormal Flag
flag.high <- final.cor.values.no.missing %>% filter(Abnormal_Flag == 'High') %>% arrange(Result.ug.dl)
dim(flag.high) # 131
table(flag.high$Reference_Units) # two ng/dL maybe wrong unit because of flag high but small value
flag.high

In [ ]:
# check high in Abnormal Flag
# check UG/ML
flag.high %>% filter(Reference_Units == 'UG/ML') # wrong unit

In [ ]:
# check high in Abnormal Flag
# check UG/DL
flag.high %>% filter(Reference_Units != 'UG/ML' & Reference_Units != 'ng/dL') %>% arrange(Result.ug.dl) # fine

In [ ]:
# check flag low
flag.low <- final.cor.values.no.missing %>% filter(Abnormal_Flag == 'Low') %>% arrange(Result.ug.dl)
dim(flag.low) # 191
table(flag.low$Reference_Units) 
flag.low

In [ ]:
# check outlier
summary(final.cor.values.no.missing$Result.ug.dl)
quantile(final.cor.values.no.missing$Result.ug.dl, c(.01,.1,.25,.50,.75,.90,.99))
hist(final.cor.values.no.missing$Result.ug.dl)

plot(x = final.cor.values.no.missing$Subject_Id, y = final.cor.values.no.missing$Result.ug.dl)

In [ ]:
# check high values after converting units
final.cor.values.no.missing %>% filter(Result.ug.dl > 100) %>% arrange(Result.ug.dl) # decide remove high than 500

In [ ]:
# remove outliers because of some high values after converting units (around 8)
cor.less.500 <- final.cor.values.no.missing %>% filter(Result.ug.dl < 500)
dim(cor.less.500)
head(cor.less.500)

In [ ]:
# remove ng/dL maybe wrong unit because of flag high but small value
table(cor.less.500$Reference_Units)

In [ ]:
cor.less.500 %>% filter(Reference_Units == 'ng/dL') # wrong unit -> discard
cor.less.500 <- cor.less.500 %>% filter(Reference_Units != 'ng/dL')
dim(cor.less.500)
table(cor.less.500$Reference_Units)

In [ ]:
# check outlier after removing
summary(cor.less.500$Result.ug.dl)
quantile(cor.less.500$Result.ug.dl, c(.01,.1,.25,.50,.75,.90,.99))
hist(cor.less.500$Result.ug.dl)

plot(x = cor.less.500$Subject_Id, y = cor.less.500$Result.ug.dl)

In [ ]:
colnames(cor.less.500)
colnames(result.w.lt)

In [ ]:
# combine result.w.lt with the cor.less.500
final.cor.raw <- rbind(cor.less.500, result.w.lt) # result.w.lt is the final file with no <
dim(final.cor.raw)
head(final.cor.raw)

In [ ]:
# check outlier after combining
summary(final.cor.raw$Result.ug.dl)
quantile(final.cor.raw$Result.ug.dl, c(.01,.1,.25,.50,.75,.90,.99))
hist(final.cor.raw$Result.ug.dl)

plot(x = final.cor.raw$Subject_Id, y = final.cor.raw$Result.ug.dl)

In [ ]:
final.cor.raw[duplicated(final.cor.raw[,c('Subject_Id', 'Seq_Date_Time',
                                            'Group_Id', 'Result', 'Reference_Units')]), ] # no dup

In [ ]:
# merge collect date by Subject_ID
dim(final.cor.raw)
final.cor.raw <- merge(final.cor.raw, data.id[,c('Subject_Id', 'Plasma_collect_date')], by = 'Subject_Id')
dim(final.cor.raw)
head(final.cor.raw)

In [ ]:
# calcualte days difference
# convert the date column to the Y-M-D format
final.cor.raw$Seq_Date <- as.Date(final.cor.raw$Seq_Date_Time, format = "%m/%d/%Y")
typeof(final.cor.raw$Seq_Date)
head(final.cor.raw$Seq_Date)
# plasma collect date
final.cor.raw$Plasma_collect_date <- as.Date(final.cor.raw$Plasma_collect_date, format = "%Y -%m -%d")
head(final.cor.raw$Plasma_collect_date)
typeof(final.cor.raw$Plasma_collect_date)

In [ ]:
head(final.cor.raw)

In [ ]:
# substract collect date and diag date
final.cor.raw["Days_Difference"] <- as.numeric(difftime(final.cor.raw$Plasma_collect_date, final.cor.raw$Seq_Date, units = "days"))
final.cor.raw["Days_Difference_Abs"] <- as.numeric(abs(final.cor.raw$Days_Difference))
typeof(final.cor.raw$Days_Difference)
typeof(final.cor.raw$Days_Difference_Abs)

In [ ]:
final.cor.raw$Cortisol_Units <- 'ug/dl'
final.cor.raw.1 <- final.cor.raw %>% select(Subject_Id, Plasma_collect_date, Seq_Date, Seq_Date_Time,
                                            Days_Difference, Days_Difference_Abs, Group_Id, 
                                            Result.ug.dl, Reference_Range, Abnormal_Flag, Cortisol_Units)
head(final.cor.raw.1)

# Cortisol

In [ ]:
# load cortisol file
cortisol <- final.cor.raw.1
dim(final.cor.raw.1)

In [ ]:
typeof(cortisol$Result.ug.dl)

In [ ]:
summary(cortisol$Result.ug.dl)

In [ ]:
dim(cortisol[duplicated(cortisol), ]) # no dup

In [ ]:
colnames(cortisol)

In [ ]:
colnames(cortisol)[8] <- 'Cortisol_value' # change column name: 'Result.ug.dl'
head(cortisol)

In [ ]:
# check duplication
dim(cortisol[duplicated(cortisol[,c('Subject_Id', 'Seq_Date', 'Cortisol_value')]), ])

In [ ]:
cortisol %>% filter(Subject_Id == '10124469' & Seq_Date == '2021-01-21')

In [ ]:
# calculate mean, median, min and max cortisol of each day per individual in final.cor.raw.selected.cols.no.dup
cortisol.stats.days <- cortisol %>% group_by(Subject_Id, Seq_Date) %>% 
                                        summarise_at(vars('Cortisol_value'), c(mean,median,min,max))
colnames(cortisol.stats.days) <- c('Subject_Id', 'Seq_Date', 
                                   'Cortisol_mean_per_day', 
                                   'Cortisol_median_per_day', 
                                   'Cortisol_min_per_day', 
                                   'Cortisol_max_per_day')
dim(cortisol.stats.days)
length(unique(cortisol.stats.days$Subject_Id))
colSums(is.na(cortisol.stats.days))
head(cortisol.stats.days)

In [ ]:
cor.date <- cortisol[,c('Subject_Id', 'Seq_Date', 'Days_Difference', 'Days_Difference_Abs')]
dim(cor.date)
dim(cor.date[duplicated(cor.date), ])
cor.date <- cor.date[!duplicated(cor.date), ]
dim(cor.date)

In [ ]:
# merge to have days difference (cortisol)
cortisol.stats.days.1 <- cortisol.stats.days %>% left_join(cor.date,  by = c('Subject_Id', 'Seq_Date'))
dim(cortisol.stats.days.1)
head(cortisol.stats.days.1)

## Cortisol: Closest to plasma collection date (before or after)

In [ ]:
# duplication values because of absolute value of days difference
# calculate min cortisol of each day per individual
cor.abs.date <- cortisol.stats.days.1 %>% group_by(Subject_Id, Days_Difference_Abs) %>% 
                       summarise_at(vars('Cortisol_min_per_day'), c(min))
colnames(cor.abs.date) <- c('Subject_Id', 'Days_Difference_Abs', 'Cortisol_min_abs_date')
dim(cor.abs.date)
head(cor.abs.date)

In [ ]:
length(unique(cor.abs.date$Subject_Id)) # how many unique IDs

In [ ]:
cor.abs.date %>% filter(Subject_Id == '10000197')
cor.abs.date %>% filter(Subject_Id == '10000201')

In [ ]:
# find closest date to collect date (before or after)
cor.closest.collect.date <- cor.abs.date %>%                                       # Get min by group
  group_by(Subject_Id) %>%
  summarise_at(vars(Days_Difference_Abs),
               list(Closest_date_collect_date_gap = min))
dim(cor.closest.collect.date)
head(cor.closest.collect.date)

colSums(is.na(cor.closest.collect.date)) # check missing value

In [ ]:
cor.abs.date %>% filter(Subject_Id == 10000201) %>% arrange(Days_Difference_Abs)

In [ ]:
# merge to have cortisol values
cor.value.closest.collect.date <- merge(cor.closest.collect.date, cor.abs.date, 
                                      by.x = c('Subject_Id', 'Closest_date_collect_date_gap'),
                                      by.y = c('Subject_Id', 'Days_Difference_Abs'))
dim(cor.value.closest.collect.date)
head(cor.value.closest.collect.date)

In [ ]:
colnames(cor.value.closest.collect.date) <- c('Subject_Id', 'Cortisol_closest_date_collect_date_gap', 
                                           'Cortisol_min_closest_measure_date_to_collect_date')
dim(cor.value.closest.collect.date)
head(cor.value.closest.collect.date)

In [ ]:
# merge to have time 
min.cortisol.closest.collect.date <- merge(cor.value.closest.collect.date, cortisol, 
                                      by.x = c('Subject_Id', 'Cortisol_closest_date_collect_date_gap', 
                                               'Cortisol_min_closest_measure_date_to_collect_date'),
                                      by.y = c('Subject_Id', 'Days_Difference_Abs', 'Cortisol_value'))

head(min.cortisol.closest.collect.date)
dim(min.cortisol.closest.collect.date)

In [ ]:
colSums(is.na(min.cortisol.closest.collect.date)) # check missing values

In [ ]:
dim(min.cortisol.closest.collect.date[duplicated(min.cortisol.closest.collect.date), ]) # no dup

In [ ]:
length(unique(min.cortisol.closest.collect.date$Subject_Id)) # one duplicates

In [ ]:
min.cortisol.closest.collect.date[min.cortisol.closest.collect.date$Subject_Id == 10009228, ] # mutiple time record

In [ ]:
# remove 1 dup
min.cortisol.closest.collect.date.final <- min.cortisol.closest.collect.date[!duplicated(min.cortisol.closest.collect.date[,c('Subject_Id', 'Seq_Date')]),]
dim(min.cortisol.closest.collect.date.final)

In [ ]:
# select cols
min.cortisol.closest.collect.date.final <- min.cortisol.closest.collect.date.final %>% 
                                                        select(Subject_Id, Cortisol_closest_date_collect_date_gap,
                                                                Cortisol_min_closest_measure_date_to_collect_date,
                                                                Seq_Date_Time,
                                                                Seq_Date,
                                                                Cortisol_Units)

In [ ]:
# change colname
colnames(min.cortisol.closest.collect.date.final) <- c('Subject_Id','Cortisol_closest_date_collect_date_gap_abs',
                                                       'Cortisol_min_value_closest_measure_date_to_collect_date',
                                                       'Cortisol_min_closest_measure_date_to_collect_date_time',
                                                       'Cortisol_min_closest_measure_date_to_collect_date',
                                                       'Cortisol_reference_units')
dim(min.cortisol.closest.collect.date.final)
head(min.cortisol.closest.collect.date.final)

## Cortisol value per ind

In [ ]:
head(cortisol.stats.days.1)

In [ ]:
# calculate median cortisol per individual
cortisol.median.ind <- cortisol.stats.days.1 %>% group_by(Subject_Id) %>% summarise_at(vars('Cortisol_median_per_day'), 
                                                                                       median)
dim(cortisol.median.ind) # only 898 have cortisol

names(cortisol.median.ind)[2] <- 'Cortisol_median_ind'

colSums(is.na(cortisol.median.ind)) # no missing
head(cortisol.median.ind)

In [ ]:
# check median Cortisol result to see outliers
summary(cortisol.median.ind$Cortisol_median_ind)
quantile(cortisol.median.ind$Cortisol_median_ind, c(.01,.1,.25,.50,.75,.90,.99))
hist(cortisol.median.ind$Cortisol_median_ind)

plot(x = cortisol.median.ind$Subject_Id, y = cortisol.median.ind$Cortisol_median_ind)

In [ ]:
# calculate mean cortisol per individual
cortisol.mean.ind <- cortisol.stats.days.1 %>% group_by(Subject_Id) %>% summarise_at(vars('Cortisol_mean_per_day'), 
                                                                                     mean)
dim(cortisol.mean.ind) # only 898 have cortisol

names(cortisol.mean.ind)[2] <- 'Cortisol_mean_ind'

colSums(is.na(cortisol.mean.ind)) # no missing
head(cortisol.mean.ind)

In [ ]:
# calculate min cortisol per individual
cortisol.min.ind <- cortisol.stats.days.1 %>% group_by(Subject_Id) %>% summarise_at(vars('Cortisol_min_per_day'), min)
dim(cortisol.min.ind) # only 898 have cortisol

names(cortisol.min.ind)[2] <- 'Cortisol_min_ind'

colSums(is.na(cortisol.min.ind)) # no missing
head(cortisol.min.ind)

In [ ]:
# calculate max cortisol per individual
cortisol.max.ind <- cortisol.stats.days.1 %>% group_by(Subject_Id) %>% summarise_at(vars('Cortisol_max_per_day'), max)
dim(cortisol.max.ind) # only 898 have cortisol

names(cortisol.max.ind)[2] <- 'Cortisol_max_ind'

colSums(is.na(cortisol.max.ind)) # no missing
head(cortisol.max.ind)

In [ ]:
cortisol.values.ind <- cortisol.mean.ind %>% left_join(cortisol.median.ind, by = 'Subject_Id') %>%
                                    left_join(cortisol.min.ind, by = 'Subject_Id') %>%
                                    left_join(cortisol.max.ind, by = 'Subject_Id')
head(cortisol.values.ind)
dim(cortisol.values.ind)

In [ ]:
# merge files
final.cortisol <- cortisol.values.ind %>% left_join(min.cortisol.closest.collect.date.final, by = "Subject_Id")
head(final.cortisol)
dim(final.cortisol)

In [ ]:
# add column: Yes or No for available cortisol value
final.cortisol$Any_Cortisol_no_ACTH_Existence_Yes_No <- 'Yes' # yes for available cortisol value
# relocate Any_Cortisol_no_ACTH_Existence_Yes_No to the second column
final.cortisol <- final.cortisol %>% relocate(Any_Cortisol_no_ACTH_Existence_Yes_No, .after = Subject_Id)
head(final.cortisol)

## Binning time of collecting cortisol

### Use min cortisol

In [ ]:
final.cortisol$Min_Cortisol_closest_collect_Date_Time <- as.POSIXct(final.cortisol$Cortisol_min_closest_measure_date_to_collect_date_time, format="%m/%d/%Y %H:%M", tz="EST")
final.cortisol$Min_Cortisol_closest_collect_Date_Only <- as.Date(final.cortisol$Min_Cortisol_closest_collect_Date_Time)
final.cortisol$Min_Cortisol_closest_collect_Hour_Only <- format(final.cortisol$Min_Cortisol_closest_collect_Date_Time, format = "%H")
final.cortisol$Min_Cortisol_closest_collect_Minute_Only <- format(final.cortisol$Min_Cortisol_closest_collect_Date_Time, format = "%M")
final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal <- as.numeric(final.cortisol$Min_Cortisol_closest_collect_Hour_Only) + as.numeric(final.cortisol$Min_Cortisol_closest_collect_Minute_Only)/60
head(final.cortisol)

In [ ]:
#binning
#4 AM - 12 PM  equivalent to 4:00 - 11:59                      <== group 1
#12 AM - 18 PM equivalent to 12:00 - 17:59                     <== group 2
#18 PM – 4 AM equivalent to 18:00 - 23:59 and 00:00 - 3:59     <== group 3

In [ ]:
final.cortisol$Min_Cortisol_closest_collect_Hour_Group <- NA

final.cortisol$Min_Cortisol_closest_collect_Hour_Group[(final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal >= 4) & (final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal < 12)] <- 1
final.cortisol$Min_Cortisol_closest_collect_Hour_Group[(final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal >= 12) & (final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal < 18)] <- 2
final.cortisol$Min_Cortisol_closest_collect_Hour_Group[(final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal >= 18) | (final.cortisol$Min_Cortisol_closest_collect_Hour_Decimal < 4)] <- 3

head(final.cortisol[final.cortisol$Min_Cortisol_closest_collect_Hour_Group == 1, ])
head(final.cortisol[final.cortisol$Min_Cortisol_closest_collect_Hour_Group == 2, ])
head(final.cortisol[final.cortisol$Min_Cortisol_closest_collect_Hour_Group == 3, ])

In [ ]:
table(final.cortisol$Min_Cortisol_closest_collect_Hour_Group) # check how many samples per time collection

In [ ]:
# draw plot
library(ggplot2)

ggplot(final.cortisol,aes(x=Cortisol_min_value_closest_measure_date_to_collect_date))+ 
geom_density()+ facet_grid(~Min_Cortisol_closest_collect_Hour_Group)

In [ ]:
# anova of time collection
res_aov_min <- aov(Cortisol_min_value_closest_measure_date_to_collect_date ~ Min_Cortisol_closest_collect_Hour_Group,
  data = final.cortisol
)

summary(res_aov_min)

In [ ]:
tapply(final.cortisol$Cortisol_min_value_closest_measure_date_to_collect_date, final.cortisol$Min_Cortisol_closest_collect_Hour_Group,
  function(x) format(summary(x)))

In [ ]:
head(final.cortisol)

### Use all cortisol

In [ ]:
head(cortisol)

In [ ]:
cortisol$Seq_Date_Time <- as.POSIXct(cortisol$Seq_Date_Time, format="%m/%d/%Y %H:%M", tz="EST")
cortisol$Seq_Date_Only <- as.Date(cortisol$Seq_Date_Time)
cortisol$Seq_Hour_Only <- format(cortisol$Seq_Date_Time, format = "%H")
cortisol$Seq_Minute_Only <- format(cortisol$Seq_Date_Time, format = "%M")
cortisol$Seq_Hour_Decimal <- as.numeric(cortisol$Seq_Hour_Only) + as.numeric(cortisol$Seq_Minute_Only)/60
head(cortisol)

In [ ]:
cortisol$Seq_Hour_Group <- NA

cortisol$Seq_Hour_Group[(cortisol$Seq_Hour_Decimal >= 4) & (cortisol$Seq_Hour_Decimal < 12)] <- 1
cortisol$Seq_Hour_Group[(cortisol$Seq_Hour_Decimal >= 12) & (cortisol$Seq_Hour_Decimal < 18)] <- 2
cortisol$Seq_Hour_Group[(cortisol$Seq_Hour_Decimal >= 18) | (cortisol$Seq_Hour_Decimal < 4)] <- 3

head(cortisol[cortisol$Seq_Hour_Group == 1, ])
head(cortisol[cortisol$Seq_Hour_Group == 2, ])
head(cortisol[cortisol$Seq_Hour_Group == 3, ])

In [ ]:
table(cortisol$Seq_Hour_Group) # check how many samples per time collection

In [ ]:
# draw plot
library(ggplot2)

ggplot(cortisol,aes(x=Cortisol_value))+ 
geom_density()+ facet_grid(~Seq_Hour_Group)

In [ ]:
# anova of time collection
res_aov_min <- aov(Cortisol_value ~ Seq_Hour_Group,
  data = cortisol
)

summary(res_aov_min)

In [ ]:
tapply(cortisol$Cortisol_value, cortisol$Seq_Hour_Group,
  function(x) format(summary(x)))